# Reversed parameter calculation.

Target system : Dispersively coupled a transmon and a resonator.

The transmon ($\hat{H}_{q}$) is approximated by Kerr or Duffing oscilltor like below,

\begin{equation}
    \hat{H}_q = (\omega_q+E_C) \hat{b}^\dagger \hat{b} - \frac{E_C}{12} \left( \hat{b} + \hat{b}^\dagger \right) ^4,
\end{equation}

or

\begin{equation}
    \hat{H}_q = \omega_q \hat{b}^\dagger \hat{b} - \frac{E_C}{2} \hat{b}^\dagger \hat{b}^\dagger \hat{b} \hat{b}.
\end{equation}

When transversely driven by a linear resonator, the total system Hamiltonian is given by,

\begin{align}
    \hat{H} & = \hat{H}_{q} + \omega_c \hat{a}^\dagger \hat{a} + g (\hat{b} + \hat{b}^\dagger) (\hat{a} + \hat{a}^\dagger).
\end{align}

In low excitation limit, for both Kerr and Duffing cases, the Hamiltonian can be reduced to,

\begin{align}
    \hat{H}_{low} & = \omega'_{q} \hat{b}^\dagger \hat{b} - \frac{A_{q}}{2} \hat{b}^\dagger \hat{b}^\dagger \hat{b} \hat{b}+ \omega'_{c} \hat{a}^\dagger \hat{a} - \frac{A_{c}}{2} \hat{a}^\dagger \hat{a}^\dagger \hat{a} \hat{a}
-2A_{qc} \hat{a}^\dagger \hat{a}\hat{b}^\dagger \hat{b}.
\end{align}


## Definition of parameters :
$\omega_{q}$  : Bare Transmon frequency \
$\omega'_{q}$ : Dressed Transmon frequency.\
$\omega_{c}$  : Bare Resonator frequency.\
$\omega'_{c}$ : Dressed resonator frequency.\
$E_c$         : Charging energy of Transmon.\
$g$           : Bare coupling \
$A_q$         : Transmon anharmonicty.\
$A_c$         : Resonator anharminicity.\
$A_{qc}$      : Cross anharmonicity.


## An objective :

This code finds the parameters of bare Hamiltonian ($\omega_{q},\omega_{c}, E_c, g$) from the parameters in low excitation limit Hamiltonian ($\omega'_{q},\omega'_{c}, A_q, A_c, A_{qc}$) and vice versa.

Requirements : \
when the $2\omega_{r}$ > $\omega_{q}$ (transmon resonant frequency) > $\omega_{r}$ (resonator resonant frequency) or \
when the $2\omega_{q}$ > $\omega_{r}$ (transmon resonant frequency) > $\omega_{q}$ (resonator resonant frequency).

### To do list : 
Kerr coefficient finding.

In [3]:
from qutip import *
from scipy.optimize import minimize
from scipy import stats
import numpy as np
import matplotlib.pyplot as plt

In [9]:
# Dress parameters 
wq_dressed = 6.4438
wc_dressed = 4.0555
A_qc = 0.000497*2 
A_q = 0.150

# Functions
Extract bare parameters from given dressed paramters.

In [18]:
def model_auto(params_bare,params_dressed, Nq = 10, Nc=10, sys='Kerr', detun='Pos'):
    """
    Calculate the cost function that represents how the given bare and dressed parameters are consistent.
    ----------------------
    Input parameters 
        params_bare  : wc, wq, g, Ec
        params_dress : wc_dressd, wq_dressd, A_q, A_qc
    Return
        Cost function.
        
    """
    wc,wq,g, Ec                     = params_bare
    wc_dressd, wq_dressd, A_q, A_qc = params_dress

    Nq = 10             # number of levels in qubit
    Nc = 10             # number of levels in resonator  
    b = tensor(destroy(Nq), qeye(Nc))
    a = tensor(qeye(Nc), destroy(Nq))
    nq = b.dag()*b
    nc = a.dag()*a
    
    if sys == 'Duffing':
        Hi = (wq+Ec)*nq + wc*nc - Ec/12*(b + b.dag())**4 
    elif sys == 'Kerr':
        Hi = (wq)*nq + wc*nc - Ec/2* b.dag()* b.dag()*b*b
    Hc = g*(a + a.dag())*(b + b.dag())  # coupling
    EE = (Hi+Hc).eigenenergies()
    if detun == 'Pos':
        wq_dressed_p = EE[2]-EE[0]
        wc_dressed_p = EE[1]-EE[0]
        A_qc_p = EE[2]-EE[0]-EE[4]+EE[1]
        A_q_p = EE[2]-EE[0]-EE[6]+EE[2]
    elif detun == 'Neg':
        wq_dressed_p = EE[1]-EE[0]
        wc_dressed_p = EE[2]-EE[0]
        A_qc_p = EE[1]-EE[0]- EE[4]+EE[2]
        A_q_p = EE[1]-EE[0]-EE[3]+EE[1]
    return (wq_dressed_p-wq_dressed)**2 + (wc_dressed_p-wc_dressed)**2 +(A_qc_p-A_qc)**2 +(A_q_p-A_q)**2
             # Cost function.

# initial bare parameters.
wc_ini = 4.0755
wq_ini  = 6.8112
g_ini = 0.12135
Ec_ini = 0.137

Nq = 10
Nc = 10
params_dressed = [wc_dressed,wq_dressed,A_q, A_qc]

# find the optimal combination bare parameters that minimize the cost function for given dressed parameters.

result = minimize(model_auto, [wc_ini,wq_ini, g_ini, Ec_ini],args=(params_dressed, Nq, Nc, 'Kerr', 'Pos'))
print('Parameter estimation \nwc :{} \nwq :{} \ng  :{} \nEc :{}'.format(result.x[0],result.x[1],result.x[2],result.x[3]))


Parameter estimation 
wc :4.060489806498666 
wq :6.440682865366634 
g  :0.09844896232787818 
Ec :0.1505202766379229


In [34]:
def reverseconfirm (result,sys='Kerr',detun='Pos'):
    """
    Calculate dressed parameters for 
    
    ---------------------------
    Input parameters 
        result  :  output of 'minimize' function. 
    Return
        wq_dressed, wc_dressed, A_qc, Aq.
        
    """
    wc = result.x[0]
    wq = result.x[1]
    g = result.x[2]
    Ec =  result.x[3]
    
    Nq = 10             # number of levels in qubit
    Nc = 10             # number of levels in resonator  
    b = tensor(destroy(Nq), qeye(Nc))
    a = tensor(qeye(Nc), destroy(Nq))
    nq = b.dag()*b
    nc = a.dag()*a
    
    if sys == 'Duffing':
        Hi = (wq+Ec)*nq + wc*nc - Ec/12*(b + b.dag())**4 
    elif sys == 'Kerr':
        Hi = (wq)*nq + wc*nc - Ec/2* b.dag()* b.dag()*b*b
    Hc = g*(a + a.dag())*(b + b.dag())  # coupling
    EE = (Hi+Hc).eigenenergies()
    if detun == 'Pos':
        wq_dressed = EE[2]-EE[0]
        wc_dressed = EE[1]-EE[0]
        Aqc = (EE[2]-EE[0]-EE[4]+EE[1])*0.5
        Aq = EE[2]-EE[0]-EE[6]+EE[2]
        print('wq_dressed :{} \nwc_dressed :{} \nA_qc       :{} \nA_q        :{}'.format(wq_dressed,wc_dressed,Aqc,Aq))
    elif detun == 'Neg':
        wq_dressed = EE[1]-EE[0]
        wc_dressed = EE[2]-EE[0]
        Aqc = (EE[1]-EE[0]- EE[4]+EE[2])*0.5
        Aq = EE[1]-EE[0]-EE[3]+EE[1]
        print('wq_dressed :{} \nwc_dressed :{} \n$A_qc$      :{}\nA_q       :{}'.format(wq_dressed,wc_dressed,Aqc,Aq))

reverseconfirm(result,sys='Kerr',detun='Pos')

wq_dressed :6.443799238432104 
wc_dressed :4.055499770375225 
A_qc       :0.00028645821257677895 
A_q        :0.15000177730528197
